Krishna Sharma | AP22110010128

In [1]:
documents = {
    1: "data algorithm structure",
    2: "database query system", 
    3: "algorithm analysis optimization",
    4: "machine learning data model",
    5: "network protocol security",
    6: "operating system process scheduling",
    7: "computer architecture hardware",
    8: "data mining machine learning",
}

BSBI (Blocked Sort-Based Indexing)

In [ ]:
def bsbi_build_index(documents):
    term_doc_pairs = []
    for doc_id, text in documents.items():
        for word in text.lower().split():
            term_doc_pairs.append((word, doc_id))
    
    print(f"Term-DocID pairs: {term_doc_pairs}")
    
    term_doc_pairs.sort(key=lambda x: x[0])
    print(f"Sorted pairs: {term_doc_pairs}")
    
    inverted_index = {}
    for term, doc_id in term_doc_pairs:
        if term not in inverted_index:
            inverted_index[term] = set()
        inverted_index[term].add(doc_id)
    
    return inverted_index

bsbi_index = bsbi_build_index(documents)
print(f"BSBI Index: {dict(sorted(bsbi_index.items()))}")

Term-DocID pairs: [('data', 1), ('algorithm', 1), ('structure', 1), ('database', 2), ('query', 2), ('system', 2), ('algorithm', 3), ('analysis', 3), ('optimization', 3), ('machine', 4), ('learning', 4), ('data', 4), ('model', 4), ('network', 5), ('protocol', 5), ('security', 5), ('operating', 6), ('system', 6), ('process', 6), ('scheduling', 6), ('computer', 7), ('architecture', 7), ('hardware', 7), ('data', 8), ('mining', 8), ('machine', 8), ('learning', 8)]
Sorted pairs: [('algorithm', 1), ('algorithm', 3), ('analysis', 3), ('architecture', 7), ('computer', 7), ('data', 1), ('data', 4), ('data', 8), ('database', 2), ('hardware', 7), ('learning', 4), ('learning', 8), ('machine', 4), ('machine', 8), ('mining', 8), ('model', 4), ('network', 5), ('operating', 6), ('optimization', 3), ('process', 6), ('protocol', 5), ('query', 2), ('scheduling', 6), ('security', 5), ('structure', 1), ('system', 2), ('system', 6)]
BSBI Index: {'algorithm': {1, 3}, 'analysis': {3}, 'architecture': {7}, 'com

SPIMI (Single-Pass In-Memory Indexing)

In [6]:
def spimi_build_index(documents):
    inverted_index = {}
    
    for doc_id, text in documents.items():
        print(f"{doc_id}: '{text}'")
        
        for word in text.lower().split():
            if word not in inverted_index:
                inverted_index[word] = set()
            inverted_index[word].add(doc_id)
        
        print(f"  Index: {dict(sorted(inverted_index.items()))}")
    
    return inverted_index

spimi_index = spimi_build_index(documents)
print(f"\nFinal SPIMI Index: {dict(sorted(spimi_index.items()))}")

1: 'data algorithm structure'
  Index: {'algorithm': {1}, 'data': {1}, 'structure': {1}}
2: 'database query system'
  Index: {'algorithm': {1}, 'data': {1}, 'database': {2}, 'query': {2}, 'structure': {1}, 'system': {2}}
3: 'algorithm analysis optimization'
  Index: {'algorithm': {1, 3}, 'analysis': {3}, 'data': {1}, 'database': {2}, 'optimization': {3}, 'query': {2}, 'structure': {1}, 'system': {2}}
4: 'machine learning data model'
  Index: {'algorithm': {1, 3}, 'analysis': {3}, 'data': {1, 4}, 'database': {2}, 'learning': {4}, 'machine': {4}, 'model': {4}, 'optimization': {3}, 'query': {2}, 'structure': {1}, 'system': {2}}
5: 'network protocol security'
  Index: {'algorithm': {1, 3}, 'analysis': {3}, 'data': {1, 4}, 'database': {2}, 'learning': {4}, 'machine': {4}, 'model': {4}, 'network': {5}, 'optimization': {3}, 'protocol': {5}, 'query': {2}, 'security': {5}, 'structure': {1}, 'system': {2}}
6: 'operating system process scheduling'
  Index: {'algorithm': {1, 3}, 'analysis': {3}, '

Boolean Query Implementation

In [7]:
def boolean_query(query, inverted_index, doc_ids):
    tokens = query.lower().split()
    if not tokens:
        return set()
    
    all_docs = set(doc_ids)
    
    if tokens[0] == 'not' and len(tokens) > 1:
        result = all_docs - inverted_index.get(tokens[1], set())
        i = 2
    else:
        result = inverted_index.get(tokens[0], set())
        i = 1
    
    while i < len(tokens):
        if tokens[i] == 'and' and i + 1 < len(tokens):
            if tokens[i + 1] == 'not' and i + 2 < len(tokens):
                term = all_docs - inverted_index.get(tokens[i + 2], set())
                i += 3
            else:
                term = inverted_index.get(tokens[i + 1], set())
                i += 2
            result = result & term
            
        elif tokens[i] == 'or' and i + 1 < len(tokens):
            if tokens[i + 1] == 'not' and i + 2 < len(tokens):
                term = all_docs - inverted_index.get(tokens[i + 2], set())
                i += 3
            else:
                term = inverted_index.get(tokens[i + 1], set())
                i += 2
            result = result | term
        else:
            i += 1
    
    return result

index = spimi_index

In [13]:
queries_1 = [
    "data AND algorithm",
    "machine OR learning", 
    "NOT security"
]

for q in queries_1:
    result = boolean_query(q, index, documents.keys())
    print(f"{q:20} -> {sorted(result)}")

data AND algorithm   -> [1]
machine OR learning  -> [4, 8]
NOT security         -> [1, 2, 3, 4, 6, 7, 8]


In [14]:
queries_2 = [
    "data AND NOT mining",
    "algorithm OR system AND data",
    "machine AND learning OR analysis"
]

for q in queries_2:
    result = boolean_query(q, index, documents.keys())
    print(f"{q:32} -> {sorted(result)}")

data AND NOT mining              -> [1, 4]
algorithm OR system AND data     -> [1]
machine AND learning OR analysis -> [3, 4, 8]


In [15]:
queries_3 = [
    "NOT algorithm AND system",
    "learning OR security AND NOT data",
    "protocol AND security OR hardware"
]

for q in queries_3:
    result = boolean_query(q, index, documents.keys())
    print(f"{q:35} -> {sorted(result)}")

NOT algorithm AND system            -> [2, 6]
learning OR security AND NOT data   -> [5]
protocol AND security OR hardware   -> [5, 7]
